In [1]:
import pandas as pd

In [2]:
import networkx as nx

In [ ]:
import networkx as nx
import re 
#import itertools
import operator
import copy
import heapq
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
import string
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt
from networkx import k_core
from networkx import core_number
import spacy
import sys
import numpy as np
from nltk.tokenize import word_tokenize,wordpunct_tokenize
import networkx.drawing
from networkx import common_neighbors
import pandas as pd
from collections import OrderedDict

In [3]:
import itertools

In [4]:
d2=pd.read_csv('output_lemma.csv')

In [5]:
d2.head()

,publish_date,headline_text,lemmatized
0,20030219,aba decides against community broadcasting lic...,"['aba', 'decides', 'against', 'community', 'br..."
1,20030219,act fire witnesses must be aware of defamation,"['act', 'fire', 'witness', 'must', 'aware', 'd..."
2,20030219,a g calls for infrastructure protection summit,"['g', 'call', 'infrastructure', 'protection', ..."
3,20030219,air nz staff in aust strike for pay rise,"['air', 'nz', 'staff', 'aust', 'strike', 'pay'..."
4,20030219,air nz strike to affect australian travellers,"['air', 'nz', 'strike', 'affect', 'australian'..."


In [6]:
text=d2['lemmatized'].to_list()

In [7]:
text=[]
for i in range(len(d2)):
    text.append(d2['lemmatized'][i].strip("',[]").replace("'", "").replace(",",'').split())

>Words-of-Graph


>Create the input for the K-core algorithm

In [8]:
def terms_to_graph_sents(text, w,stopping_end_of_line=0):
    from_to = {}
    if(not stopping_end_of_line):
        extended_text=[]
        for sublist in text:
            extended_text.extend(sublist)
        text=[extended_text]
    for k,sents in enumerate(text):
        len_sents=len(sents)

        # create initial complete graph (first w terms)
        terms_temp = sents[0:min(w,len_sents)]
        indexes = list(itertools.combinations(range(min(w,len_sents)), r=2))
        new_edges = []
        for my_tuple in indexes:
            new_edges.append(tuple([terms_temp[i] for i in my_tuple]))

        for new_edge in new_edges:
            if new_edge in from_to:
                from_to[new_edge] += 1
            else:
                from_to[new_edge] = 1

        if(w<=len_sents):
            # then iterate over the remaining terms
            for i in range(w, len_sents):
                considered_term = sents[i] # term to consider
                terms_temp = sents[(i-w+1):(i+1)] # all terms within sliding window

                # edges to try
                candidate_edges = []
                for p in range(w-1):
                    candidate_edges.append((terms_temp[p],considered_term))

                for try_edge in candidate_edges:

                    if try_edge[1] != try_edge[0]:
                    # if not self-edge

                        # if edge has already been seen, update its weight
                        if try_edge in from_to:
                            from_to[try_edge] += 1

                        # if edge has never been seen, create it and assign it a unit weight     
                        else:
                            from_to[try_edge] = 1
    
    return(from_to)


In [9]:
tuples_words_sents_unweighted=list(terms_to_graph_sents(text, w=5,stopping_end_of_line=1).keys())

In [10]:
tuples_words_sents_weighted=terms_to_graph_sents(text, w=5,stopping_end_of_line=0)

In [11]:
print("tuples unweighted \n")
tuples_words_sents_unweighted[:10]

tuples unweighted 



[('aba', 'decides'),
 ('aba', 'against'),
 ('aba', 'community'),
 ('aba', 'broadcasting'),
 ('decides', 'against'),
 ('decides', 'community'),
 ('decides', 'broadcasting'),
 ('against', 'community'),
 ('against', 'broadcasting'),
 ('community', 'broadcasting')]

In [12]:
tuples_words_sents_weighted

{('aba', 'decides'): 1,
 ('aba', 'against'): 1,
 ('aba', 'community'): 2,
 ('aba', 'broadcasting'): 1,
 ('decides', 'against'): 36,
 ('decides', 'community'): 2,
 ('decides', 'broadcasting'): 1,
 ('against', 'community'): 24,
 ('against', 'broadcasting'): 1,
 ('community', 'broadcasting'): 1,
 ('decides', 'licence'): 1,
 ('against', 'licence'): 9,
 ('community', 'licence'): 10,
 ('broadcasting', 'licence'): 1,
 ('against', 'act'): 67,
 ('community', 'act'): 25,
 ('broadcasting', 'act'): 1,
 ('licence', 'act'): 8,
 ('community', 'fire'): 72,
 ('broadcasting', 'fire'): 1,
 ('licence', 'fire'): 11,
 ('act', 'fire'): 93,
 ('broadcasting', 'witness'): 1,
 ('licence', 'witness'): 4,
 ('act', 'witness'): 6,
 ('fire', 'witness'): 18,
 ('licence', 'must'): 1,
 ('act', 'must'): 6,
 ('fire', 'must'): 11,
 ('witness', 'must'): 1,
 ('act', 'aware'): 2,
 ('fire', 'aware'): 4,
 ('witness', 'aware'): 1,
 ('must', 'aware'): 1,
 ('fire', 'defamation'): 2,
 ('witness', 'defamation'): 2,
 ('must', 'defama

In [13]:
nb_iter=0
print("tuples weighted \n")
for key in tuples_words_sents_weighted.keys():
    if(nb_iter <10):
        nb_iter+=1
        print(key,tuples_words_sents_weighted[key])

tuples weighted 

('aba', 'decides') 1
('aba', 'against') 1
('aba', 'community') 2
('aba', 'broadcasting') 1
('decides', 'against') 36
('decides', 'community') 2
('decides', 'broadcasting') 1
('against', 'community') 24
('against', 'broadcasting') 1
('community', 'broadcasting') 1


In [14]:
def unweighted_graph(tuples_words_sents_unweighted):
    G = nx.Graph()
    G.add_edges_from(tuples_words_sents_unweighted)
    return(G)


In [15]:
G_unweighted=unweighted_graph(tuples_words_sents_unweighted)

In [ ]:
## UNWEIGHTED
nx.draw(G_unweighted, with_labels=True,font_size=0.02,width=1,node_size=10,label='Graph Unweighted GOT')
plt.show()